In [1]:
# Al tener los datos subidos en el One Drive personal debemos de llamarlos y
# guardarlos en el entorno de Google Colab

from google.colab import drive # Llamamos a la librería necesaria

drive.mount('/gdrive') # Monta el drive en Google Colab y establece la ruta de acceso.
%cp -r /gdrive/MyDrive/UEM/data/TFM/* /content/ # Cogemos todos los datos de la ruta

Mounted at /gdrive


In [2]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
import os

In [3]:
# Abre el archivo de entrenamiento
with open('./IDC_train.dat', 'rb') as f:
    (X_train, y_train) = pickle.load(f)
# Abre el archivo de prueba
with open('./IDC_test.dat', 'rb') as f:
    (X_test, y_test) = pickle.load(f)

# Comprobamos la cantidad de datos que tenemos
print(X_train.shape)
print(y_train.shape)

(222019, 50, 50, 3)
(222019,)


In [4]:
from sklearn.model_selection import train_test_split

# Obtiene el 30% de los datos de entrenamiento
X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Obtiene el 30% de los datos de prueba
X_test_subset, _, y_test_subset, _ = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

# Imprime las formas de los conjuntos reducidos
print("Forma de X_train_subset:", X_train_subset.shape)
print("Forma de y_train_subset:", y_train_subset.shape)
print("Forma de X_test_subset:", X_test_subset.shape)
print("Forma de y_test_subset:", y_test_subset.shape)

Forma de X_train_subset: (155413, 50, 50, 3)
Forma de y_train_subset: (155413,)
Forma de X_test_subset: (38853, 50, 50, 3)
Forma de y_test_subset: (38853,)


In [5]:
import pickle
from keras.optimizers import Adam
from keras.layers import Flatten, Dense
from keras.models import Model
from tensorflow.keras.applications import ResNet50

class BreastCancerModel:
    def __init__(self, name, base_model, input_shape):
        self.name = name
        self.base_model = base_model
        self.input_shape = input_shape
        self.models = {}

    def build_model(self, num_neurons):
        x = Flatten()(self.base_model.output)
        x = Dense(num_neurons, activation='relu')(x)
        x = Dense(num_neurons // 2, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        model = Model(self.base_model.input, predictions)
        return model

    def train(self, X_train, y_train, X_val, y_val, batch_size, epochs, num_neurons):
        model = self.build_model(num_neurons)
        model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_val, y_val))
        self.models[num_neurons] = model
        return history

    def save_model(self, num_neurons, filepath):
        if num_neurons in self.models:
            model = self.models[num_neurons]
            model.save(filepath)
            print("Model saved successfully.")
        else:
            print("Model not found for the specified number of neurons.")

# Define the input shape
input_shape = (50, 50, 3)  # Adjust the number of channels if necessary

# Create ResNet50 model
resnet_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
resnet_model = BreastCancerModel("ResNet50", resnet_base_model, input_shape)

# Train the models
batch_size = 32
epochs = 10

# Obtener el directorio de Google Drive
drive_path = '/gdrive/MyDrive/UEM/data/TFM/'

# Ruta de la carpeta donde deseas guardar los modelos
folder_path = os.path.join(drive_path, 'Modelos/ResNet50/')
os.makedirs(folder_path, exist_ok=True)  # Crear la carpeta si no existe

# Train and save models with different numbers of neurons
num_neurons_list = [10, 20, 30]
for num_neurons in num_neurons_list:
    history = resnet_model.train(X_train_subset, y_train_subset, X_test_subset, y_test_subset, batch_size, epochs, num_neurons)

    # Guardar el historial (history)
    history_filepath = os.path.join(folder_path, f"ResNet50_{num_neurons}_history.pkl")
    with open(history_filepath, 'wb') as file:
        pickle.dump(history.history, file)

    # Guarda el modelo completo
    model_filepath = os.path.join(folder_path, f"ResNet50__{num_neurons}.h5")
    resnet_model.save_model(num_neurons, model_filepath)

94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/10
4857/4857 [==============================] - 253s 49ms/step - loss: 0.3838 - accuracy: 0.8367 - val_loss: 0.5095 - val_accuracy: 0.8304
Epoch 2/10
4857/4857 [==============================] - 238s 49ms/step - loss: 0.3547 - accuracy: 0.8504 - val_loss: 0.8527 - val_accuracy: 0.8409
Epoch 3/10
4857/4857 [==============================] - 239s 49ms/step - loss: 0.3457 - accuracy: 0.8552 - val_loss: 0.3664 - val_accuracy: 0.8453
Epoch 4/10
4857/4857 [==============================] - 241s 50ms/step - loss: 0.3285 - accuracy: 0.8603 - val_loss: 0.3053 - val_accuracy: 0.8704
Epoch 5/10
4857/4857 [==============================] - 243s 50ms/step - loss: 0.3335 - accuracy: 0.8584 - val_loss: 0.3918 - val_accuracy: 0.8356
Epoch 6/10
4857/4857 [==============================] - 243s 50ms/step - loss: 0.3151 - accuracy: 0.8664 - val_loss: 0.7003 - val_accuracy: 0.7198
Epoch 7/10
4857/4857 [==============================]